#  Make Tournament CSV

In [1]:
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import requests

def get_data(player_id, page, PER_PAGE=490):
    # URL of the GraphQL endpoint
    url = 'https://api.start.gg/gql/alpha'
    
    # The GraphQL query
    query = """
      query PlayerQuery {
      player(id: "{player_id}") {
        id
        gamerTag
        sets(page: {page}, perPage: 490) {
          nodes {
            event {
              tournament {
                slug
                startAt
              }
            }
          }
        }
      }
    }""".replace('{page}', str(page)).replace('{player_id}', str(player_id))
    
    # Headers (if needed, e.g., for authentication)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer <TOKEN>'  # if authentication is needed
    }
    
    # Sending the request
    response = requests.post(url, json={'query': query}, headers=headers)
    
    result = response.json()

    return result


def get_events(result):
    events = result['data']['player']['sets']['nodes']
    events = [event['event']['tournament'] for event in events]
    return events

def get_datetime(timestamp):
    # Convert timestamp to datetime object
    datetime_object = datetime.fromtimestamp(timestamp)
    
    # Format datetime object as a readable string
    readable_date_time = datetime_object.strftime("%Y-%m-%d %H:%M:%S")
    
    return readable_date_time

def make_df(gamertag, events):
    df = pd.DataFrame(events)
    df = df.drop_duplicates().reset_index(drop=True)
    df.startAt = df.startAt.map(lambda ts: get_datetime(ts))
    df['gamertag'] = gamertag
    reordered_df = df[['gamertag', 'slug', 'startAt']]
    return reordered_df

def get_all_events(player_id):
    page = 1
    all_events = []
    while True:
        # print('page', page)
        result = get_data(player_id, page)
        events = get_events(result)
        if events == []:
            break
        all_events.extend(events)
        page += 1
    return all_events

players = [
    {"gamertag": "Tweek", "id": 15768},
    {"gamertag": "MkLeo", "id": 222927},
    {"gamertag": "Sparg0", "id": 158026},
    {"gamertag": "Light", "id": 158871},
    {"gamertag": "あcola", "id": 2691639},
    {"gamertag": "Glutonny", "id": 6122},
    {"gamertag": "Riddles", "id": 160464},
    {"gamertag": "Tea", "id": 695882},
    {"gamertag": "Kurama", "id": 175623},
]

event_df = pd.DataFrame(columns=['gamertag', 'slug', 'startAt'])
for player in players:
    events = get_all_events(player['id'])
    df = make_df(player['gamertag'], events)
    event_df = pd.concat([event_df, df]).reset_index(drop=True)

event_df

,gamertag,slug,startAt
0,Tweek,tournament/momocon-2024-7,2024-05-24 14:00:00
1,Tweek,tournament/get-on-my-level-x-canadian-fighting...,2024-05-17 10:00:00
2,Tweek,tournament/lvl-up-expo-2024,2024-04-26 12:00:00
3,Tweek,tournament/kroger-gaming-presents-the-luminosi...,2024-04-20 10:30:00
4,Tweek,tournament/kawaii-kon-2024,2024-03-29 16:00:00
...,...,...,...
2612,Kurama,tournament/gamerz-rebooted-32,2017-05-26 15:30:00
2613,Kurama,tournament/gamerz-rebooted-29,2017-05-05 22:30:00
2614,Kurama,tournament/gamerz-rebooted-28,2017-04-28 22:30:00
2615,Kurama,tournament/gamerz-rebooted-26,2017-04-14 15:30:00


# Save Tournaments CSV

In [2]:
event_df.to_csv('tournaments.csv', index=False)

# Find Most Common Tournaments

In [3]:
event_df.groupby(['slug', 'startAt']).size().sort_values(ascending=False)

slug                                             startAt            
tournament/port-priority-8-5                     2023-11-10 15:00:00    9
tournament/ludwig-smash-invitational             2022-10-21 13:00:00    9
tournament/genesis-9-1                           2023-01-20 12:00:00    9
tournament/double-down-2022                      2022-07-08 07:00:00    8
tournament/genesis-x                             2024-02-16 13:00:00    8
                                                                       ..
tournament/icarus-v                              2019-04-27 04:00:00    1
tournament/icarus-2023                           2023-06-03 03:00:00    1
tournament/ibp-masters-showdown                  2017-11-11 15:00:00    1
tournament/ib-games-4-the-end-of-sisqui-s-reign  2023-06-01 12:00:00    1
tournament/ya-es-viernes-3-bici-weekly-1         2020-07-10 23:30:00    1
Length: 2031, dtype: int64